Euclid GC Fisher Matrix Code

Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/workspace


In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

kpiv=0.05

gamma=0.545

c=3e5

print om0

0.32


In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

In [33]:
#Construct P(k,z=0) interpolating function [only Mpc units, no hubble factors]
Pkz0 = interp1d(kh*hubble, pk[0]/hubble**3)

In [34]:
#get fiducial σ8 (at z=0)
sig8 = results.get_sigma8()[0]
print sig8

0.830287480109


In [38]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001*hubble
kmax = 0.2*hubble

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [39]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print rcom(ztest)
print DA(ztest)

2596.15131772
1527.14783395


In [40]:
#LCDM growth rate
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.822099034946


In [41]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest), Dgz(0)

0.693913449649 1.0


In [42]:
#def sig8z(zc):
#    return Dgz(zc)*sig8

#print sig8z(ztest)

In [43]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg+fg(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [44]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [45]:
def photoz(kk,mu,zc):
    sigz = 0.001*(1+zc)
    return 1.0 #np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))

print photoz(0.5,0.5,ztest)

1.0


In [46]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return 1/nbar

#effective volume
def Veff(kk,mu,zc):
    return Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)+Pshot(zc)))**2

print ztest, "%.4g" %  Vsur(ztest)

0.7 9.184e+09


In [47]:
#Fisher matrix parameters 
params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps"]

In [58]:
#Fisher matrix derivatives

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg/(bg+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dPs(kk,mu,zc):#We want Pgg in [Mpc^3/h^3] units for the Euclid GC comparison
    return 1.0/(Pgg(kk,mu,zc)*hubble**3*photoz(kk,mu,zc))

In [59]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [60]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [61]:
mugrid = np.linspace(-1., 1., 200) 

In [62]:
#%%time
#   Fisher matrix   # 

Npar = len(params)
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    
    dVdz = (4*pi/3)*((1+zmax)**3*DA(zmax)**3-(1+zmin)**3*DA(zmin)**3)/(4*pi*pow(180/pi,2))
    nbar = Dzbin*dn3[zi]/dVdz
    
    kgrid = np.linspace(kmin, kmax, 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnH(kk,mu,zc)
            elif i==1:  return dlnP_dlnDA(kk,mu,zc)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==4:  return dlnP_dPs(kk,mu,zc)
            else: print "Error: index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnH(kk,mu,zc)
                    elif j==1:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==4:  return dlnP_dPs(kk,mu,zc)
                    else: print "Error: index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
            if zi==0: print i,j,Fishermat[i][j]
    #print "z =",zc,"lnDA error =", np.sqrt(linalg.inv(Fishermat)[2,2]),\
    #"lnH error =", np.sqrt(linalg.inv(Fishermat)[3,3]),\
    #"Ps error =",np.sqrt(linalg.inv(Fishermat)[4,4])

0 0 95040.7285501
0 1 -45445.1587533
0 2 22275.3386176
0 3 152160.761107
0 4 35.5276321211
1 0 -45445.1587533
1 1 97648.9744925
1 2 -45342.4229204
1 3 -125007.966219
1 4 -24.7359259727
2 0 22275.3386176
2 1 -45342.4229204
2 2 26448.3840278
2 3 63324.3957113
2 4 12.445703297
3 0 152160.761107
3 1 -125007.966219
3 2 63324.3957113
3 3 303702.942819
3 4 69.1121233838
4 0 35.5276321211
4 1 -24.7359259727
4 2 12.445703297
4 3 69.1121233838
4 4 0.0186716375833
